In [1]:
pip install torch transformers torchaudio librosa


  Using cached torchaudio-2.5.1-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached soundfile-0.13.0-py2.py3-none-win_amd64.whl.metadata (16 kB)
  Using cached soxr-0.5.0.post1-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
Using cached torchaudio-2.5.1-cp311-cp311-win_amd64.whl (2.4 MB)
Using cached torch-2.5.1-cp311-cp311-win_amd64.whl (203.1 MB)
Using cached librosa-0.10.2.post1-py3-none-any.whl (260 kB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 19.4 MB/s eta 0:00:00
Using cached soundfile-0.13.0-py2.py3-none-win_amd64.whl (1.0 MB)
Using cached soxr-0.5.0.post1-cp311-cp311-win_amd64.whl (166 kB)
   ---------------------------------------- 0.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openplayground 0.1.5 requires huggingface_hub<0.14.0,>=0.13.2, but you have huggingface-hub 0.27.1 which is incompatible.
torchvision 0.20.0 requires torch==2.5.0, but you have torch 2.5.1 which is incompatible.


In [6]:
!pip install torch torchvision torchaudio



^C


In [3]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio

# Path to the extracted model directory
model_dir = "./whisper-darija"  # Update this path if necessary

# Load the fine-tuned model and processor
processor = WhisperProcessor.from_pretrained(model_dir)
model = WhisperForConditionalGeneration.from_pretrained(model_dir)

print("Model and processor loaded successfully!")


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model and processor loaded successfully!


In [9]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent

def trim_silence(audio_path):
    audio = AudioSegment.from_file(audio_path, format="wav")
    nonsilent_ranges = detect_nonsilent(audio, min_silence_len=500, silence_thresh=-40)
    if nonsilent_ranges:
        start, end = nonsilent_ranges[0][0], nonsilent_ranges[-1][1]
        return audio[start:end]
    return audio

trimmed_audio = trim_silence("mic_ali.wav")
trimmed_audio.export("trimmed_ali.wav", format="wav")


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


<_io.BufferedRandom name='trimmed_ali.wav'>

In [10]:
import torch
# Load and preprocess the audio
audio_path = "trimmed_ali.wav"  # Update the path to your audio file
waveform, sample_rate = torchaudio.load(audio_path)

# Resample the audio to 16 kHz (required by Whisper)
if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)

# Process the audio with the processor
inputs = processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt")

# Transcribe using the fine-tuned model
with torch.no_grad():
    predicted_ids = model.generate(
        inputs.input_features,
        forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
    )

# Decode the transcription
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print(f"Transcription: {transcription}")


Transcription:  اسلام عليكم انا اعلي اصواب وصاحبي احمد بن براهيم ومحمد عيسام والاستاذ حاكي محافيدي حاولنا نفينت وينت بيرو واحد الموديل كيف حمد دري جديدنا وكي يقدر يعطينا الكتابه جلش نوكي لنا بطبت وهذه شيء مزيان وانا احنا افرحانين بغينا رقم زيانه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا فريقه جدا ف
